#### The goal of this project file is to predict flow using ppt data

Version 4: 
- instead of systematically adding/removing each variable to achieve the best r2 values, we are randomly selecting a set of predictors (n = 4, 5, 6....10, etc.)

In [ ]:
import pandas as pd
import math
import numpy as np
import sklearn as sk
import sklearn.datasets as skd
import sklearn.ensemble as ske
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from pprint import pprint
import random
import xlsxwriter
from openpyxl.styles import Font
from openpyxl import Workbook

%matplotlib inline

In [ ]:
# importing flow dataset
df_flow = pd.read_excel('Daily ISCO flow and precipitation.xlsx', sheet_name = 'Sheet1')


df_flow = df_flow.drop(columns=['Daily precip (in)','Daily precip (mm)','S12-ISCO flow (cms)','T12-ISCO flow (cms)'])

df_flow = df_flow.rename(columns={'T8-ISCO flow (cms)':'Flow8', 'S11-ISCO flow (cms)':'Flow11', 'S12+T12':'Flow12'})

#df_flow = pd.DataFrame(df_flow, columns=['Sample date', 'Sample type', 'Site', 'Flow'])
df_flow.head()

In [ ]:
# importing weather dataset
df_weather = pd.read_excel('Weather data.xlsx', sheet_name = 'Weather')
df_weather = df_weather.rename(columns={'ppt (mm)':'ppt','tmin (degrees C)':'tmin','tmean (degrees C)':'tmean',
                                        'tmax (degrees C)':'tmax', 'tdmean (degrees C)':'tdew', 'vpdmin (hPa)': 'vpdmin', 
                                        'vpdmax (hPa)': 'vpdmax'})

df_weather['Year'] = pd.DatetimeIndex(df_weather['Date']).year
df_weather['Month'] = pd.DatetimeIndex(df_weather['Date']).month
df_weather['Day'] = pd.DatetimeIndex(df_weather['Date']).day
df_weather['Day of year'] = df_weather['Date'].dt.dayofyear

# calculate antecedent ppt and temperature
# 2-day antecedent cumulative ppt (mm)
df_weather['2-day ppt'] = round(df_weather.iloc[:,1].rolling(window=2).sum(), 1)

# 3-day antecedent cumulative ppt (mm)
df_weather['3-day ppt'] = round(df_weather.iloc[:,1].rolling(window=3).sum(), 1)

# 5-day antecedent cumulative ppt (mm)
df_weather['5-day ppt'] = round(df_weather.iloc[:,1].rolling(window=5).sum(), 1)

# 7-day antecedent cumulative ppt (mm)
df_weather['7-day ppt'] = round(df_weather.iloc[:,1].rolling(window=7).sum(), 1)

# 15-day antecedent cumulative ppt (mm)
df_weather['15-day ppt'] = round(df_weather.iloc[:,1].rolling(window=15).sum(), 1)

# 30-day antecedent cumulative ppt (mm)
df_weather['30-day ppt'] = round(df_weather.iloc[:,1].rolling(window=30).sum(), 1)

# 90-day antecedent cumulative ppt (mm)
df_weather['90-day ppt'] = round(df_weather.iloc[:,1].rolling(window=90).sum(), 1)

# 180-day antecedent cumulative ppt (mm)
df_weather['180-day ppt'] = round(df_weather.iloc[:,1].rolling(window=180).sum(), 1)

# 360-day antecedent cumulative ppt (mm)
df_weather['360-day ppt'] = round(df_weather.iloc[:,1].rolling(window=360).sum(), 1)

# annual and prev annual cumulative ppt (mm)
df_annual_ppt = df_weather.groupby(['Year'])['ppt'].sum()
df_annual_ppt = df_annual_ppt.to_frame().reset_index()
df_annual_ppt = df_annual_ppt.rename(columns={'ppt':'Annual ppt'})
df_annual_ppt['Prev annual ppt'] = df_annual_ppt['Annual ppt'].shift(1)
df_weather = pd.merge(df_weather, df_annual_ppt, left_on='Year', right_on='Year', how='left')


# 2-day antecedent avg temperature (ºC)
df_weather['2-day temp'] = round(df_weather.iloc[:,3].rolling(window=2).mean(), 1)

# 3-day antecedent avg temperature (ºC)
df_weather['3-day temp'] = round(df_weather.iloc[:,3].rolling(window=3).mean(), 1)

# 5-day antecedent avg temperature (ºC)
df_weather['5-day temp'] = round(df_weather.iloc[:,3].rolling(window=5).mean(), 1)

# 7-day antecedent avg temperature (ºC)
df_weather['7-day temp'] = round(df_weather.iloc[:,3].rolling(window=7).mean(), 1)

# 15-day antecedent avg temperature (ºC)
df_weather['15-day temp'] = round(df_weather.iloc[:,3].rolling(window=15).mean(), 1)

# 30-day antecedent avg temperature (ºC)
df_weather['30-day temp'] = round(df_weather.iloc[:,3].rolling(window=30).mean(), 1)

df_weather = df_weather.drop(columns=['tmin','tmax','tdew','vpdmin','vpdmax'])
df_train = df_weather[df_weather["Year"].isin([2015, 2016, 2017])]

#df_weather.head()

# merging flow and weather datasets
df_merged = pd.merge(df_train, df_flow, left_on='Date', right_on='Date', how='right')
df_merged = df_merged.dropna()

# write dataframe to excel
#writer = pd.ExcelWriter('Weather data processed.xlsx')
#df_merged.to_excel(writer)
#writer.save()

#df_merged.tail()
#df_merged.describe()
print(df_merged.columns)

### Random forest

In [ ]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# prompt user input to identify which subwatershed to work on
print ('which site to analyze?')
print ('Options are: 8, 11, 12')
sub = input()
print ('subwatershed ', sub, ' is selected')

sub_flow = 'Flow' + sub
y = df_merged[sub_flow]

# convert Sample date from timestamp to numeric because sklearn cannot process timestamp format
df_merged['Date'] = pd.to_numeric(pd.to_datetime(df_merged['Date']))

# look up all the variables, and remove uncessary variables
all_variables = list(df_merged.columns)
unwanted = {'Date', 'Year', 'Flow8', 'Flow11', 'Flow12'}
all_variables = [e for e in all_variables if e not in unwanted]

# create an empty list
variables = []
good_accuracy = float('-inf')
better_accuracy = float('-inf')
best_accuracy = float('-inf')
r2_prev = 0
r2 = -1
good_r2 = 0
better_r2 = 0
best_r2 = 0
good_var = 0
better_var = 0
best_var = 0
good_avg_error = float('-inf')
better_avg_error = float('-inf')
best_avg_error = float('-inf')
good_med_error = float('-inf')
better_med_error = float('-inf')
best_med_error = float('-inf')
good_df_predict = pd.DataFrame()
better_df_predict = pd.DataFrame()
best_df_predict = pd.DataFrame()

# set min and max numbers of variables in the prediction model
min_var_no = 6 
max_var_no = 12

# this is for random combination approach - set number of random sets of combination of variables in the prediction model
combo_no = 1000

for var_no in range (min_var_no, max_var_no):
    
    count = 0    
    while r2 < 0.7 and count < combo_no:
        class color:
            BLUE = '\033[94m'
            BOLD = '\033[1m'
            END = '\033[0m'
        print(color.BOLD + color.BLUE + 'NEW LOOP' + color.END)
        
        # randomly select variables
        variables = random.sample(all_variables, var_no)
        
        X = df_merged[variables]
        X_train, X_test, Y_train, Y_test = train_test_split (X, y, test_size = 0.2, random_state = 0)
        rf_base = ske.RandomForestRegressor(n_estimators = 1000, random_state = 0)
        rf_base.fit(X_train, Y_train)
        Y_pred = rf_base.predict(X_test)

        #print('training period explained variance:', round(explained_variance_score(Y_test, Y_pred),2))
        #print('training period mean abs error:', round(mean_absolute_error(Y_test, Y_pred),2))
        #print('training period mean squared error:', round(mean_squared_error(Y_test, Y_pred),2))
        #print('training period r2:', round(r2_score(Y_test, Y_pred),2))

        fet_ind = np.argsort(rf_base.feature_importances_)[::-1]
        fet_imp = rf_base.feature_importances_[np.argsort(rf_base.feature_importances_)][::-1]

        fig, ax = plt.subplots(1, 1, figsize=(8,3))
        labels = np.asarray(X_train.columns[fet_ind])
        pd.Series(fet_imp, index = labels).plot('bar', ax=ax)
        ax.set_title('Feature importance')
        plt.show()

        df_weathersub = df_weather[df_weather["Year"].isin([2018])]
        df_weathersub = df_weathersub.reset_index(drop=True)
        df_test = df_weathersub
        df_test = df_test[variables]
        df_test = df_test.dropna()
        df_test = df_test.reset_index(drop=True)

        # If 'Date' variable is present, convert the date from timestamp to numeric because sklearn cannot process timestamp format
        if 'Date' in list(df_test):
            df_test['Date'] = pd.to_numeric(pd.to_datetime(df_test['Date']))

        # use base or random depending on model performance
        #Y_pred = rf_base.predict(df_test)
        Y_pred = rf_base.predict(df_test)

        df_predict = pd.DataFrame(Y_pred, columns=['est. flow'])
        df_predict['est. flow'] = round(df_predict['est. flow'], 3)
        df_predict = pd.merge(df_test, df_predict, left_index=True, right_index=True)
        df_predict = pd.merge(df_weathersub['Date'], df_predict, left_index=True, right_index=True)
        df_predict = df_predict.rename(columns={'Date_x':'Date', 'Date_y':'Date num'})

        #df_predict['Date'] = pd.to_datetime(df_predict['Date'])

        # this is ISCO daily avg flow
        df_actual = df_flow[['Date', sub_flow]]
        df_actual['Year'] = pd.DatetimeIndex(df_actual['Date']).year
        df_actual = df_actual[df_actual["Year"].isin([2018])]
        df_actual = df_actual.dropna()

        df_predict = pd.merge(df_predict, df_actual, left_on='Date', right_on='Date', how='right')
        df_predict['error (%)'] = round(((df_predict[sub_flow]-df_predict['est. flow'])/df_predict[sub_flow]*100), 0)

        plt.plot(df_predict['Date'], df_predict['est. flow'], linestyle = 'dotted', label='estimated')
        plt.xlabel('Date')
        plt.xticks(rotation = 30)
        plt.plot(df_predict['Date'], df_predict[sub_flow], linestyle = 'dotted', label='measured')
        plt.ylabel('Estimated flow (cms)')
        plt.legend()
        plt.show()

        #print('prediction period explained variance:',round(explained_variance_score(df_predict[sub_flow], df_predict['est. flow']),2))
        #print('prediction period mean abs error:', round(mean_absolute_error(df_predict[sub_flow], df_predict['est. flow']),2))
        #print('prediction period mean squared error:', round(mean_squared_error(df_predict[sub_flow], df_predict['est. flow']),2))
        #print('prediction period r2:', round(r2_score(df_predict[sub_flow], df_predict['est. flow']),2))

        plt.boxplot(df_predict['error (%)'])
        plt.ylabel('Error in est. flow (%)')
        plt.show()
                
        r2 = round(r2_score(df_predict[sub_flow], df_predict['est. flow']),2)
        abs_error = abs(df_predict[sub_flow] - df_predict['est. flow'])
        # ignore infinity value (when anything divided by 0 cfs flow = infinity) when calculating mean
        pe = abs_error / df_predict[sub_flow]
        mape = 100* np.mean(pe[np.isfinite(pe)])
        accuracy = round(100 - mape, 1)        
        avg_error = round(df_predict['error (%)'].mean(),2)
        med_error = round(df_predict['error (%)'].median(),2) 
                   
        print('R2 value: ', r2)
        print('Median accuracy: ', accuracy, '%')
        print('Variables included:', variables)
        print('Average error:', round(df_predict['error (%)'].mean(),2))
        print('Median error:', round(df_predict['error (%)'].median(),2))     
           
        # store the best variables and its model outputs (based on r2)
        # consider setting the "best" variables based on other parameters, such as accuracy and r2
        if r2 >= good_r2:
            good_var = variables
            good_r2 = r2
            good_accuracy = accuracy
            good_avg_error = avg_error
            good_med_error = med_error           
            good_df_predict = df_predict
           
            if good_r2 >= better_r2:
                prev_better_var = better_var
                better_var = good_var
                good_var = prev_better_var
                
                prev_better_r2 = better_r2
                better_r2 = good_r2
                good_r2 = prev_better_r2
                
                prev_better_accuracy = better_accuracy
                better_accuracy = good_accuracy
                good_accuracy = prev_better_accuracy
                
                prev_better_avg_error = better_avg_error
                better_avg_error = good_avg_error
                good_avg_error = prev_better_avg_error
                
                prev_better_med_error = better_med_error
                better_med_error = good_med_error
                good_med_error = prev_better_med_error
                
                prev_better_df_predict = better_df_predict
                better_df_predict = good_df_predict
                good_df_predict = prev_better_df_predict
                
                if better_r2 >= best_r2:
                    prev_best_var = best_var
                    best_var = better_var
                    better_var = prev_best_var

                    prev_best_r2 = best_r2
                    best_r2 = better_r2
                    better_r2 = prev_best_r2

                    prev_best_accuracy = best_accuracy
                    best_accuracy = better_accuracy
                    better_accuracy = prev_best_accuracy

                    prev_best_avg_error = best_avg_error
                    best_avg_error = better_avg_error
                    better_avg_error = prev_best_avg_error

                    prev_best_med_error = best_med_error
                    best_med_error = better_med_error
                    better_med_error = prev_best_med_error

                    prev_best_df_predict = best_df_predict
                    best_df_predict = better_df_predict
                    better_df_predict = prev_best_df_predict

        r2_prev = r2
        count = count + 1
        print('no. of loop: ', count)
    
    print('no. of variable: ', var_no)

# set the excel worksheets and figures export destination to Output folder
save_path = "Outputs/"     
    
class color:
    BLUE = '\033[94m'
    BOLD = '\033[1m'
    END = '\033[0m'
print(color.BOLD + color.BLUE + 'RESULTS' + color.END)

print('GOOD')

plt.plot(good_df_predict['Date'], good_df_predict['est. flow'], linestyle = 'dotted', label='estimated')
plt.xlabel('Date')
plt.xticks(rotation = 30)
plt.plot(good_df_predict['Date'], good_df_predict[sub_flow], linestyle = 'dotted', label='measured')
plt.ylabel('Estimated flow (cms)')
plt.legend()
plt.title('Good')
plt.savefig(save_path + 'Est. vs measured flow - ' + sub + ' good.png')
plt.show()

plt.boxplot(good_df_predict['error (%)'])
plt.ylabel('Error in est. flow (%)')
plt.title('Good')
plt.savefig(save_path + 'Est. error - ' + sub + ' good.png')
plt.show()

print('Good variables: ', good_var)
print('Good r2: ', good_r2)
print('Good accuracy: ', good_accuracy, '%')
print('Good avg error: ', good_avg_error, '%')
print('Good median error: ', good_med_error, '%')

print('BETTER')

plt.plot(better_df_predict['Date'], better_df_predict['est. flow'], linestyle = 'dotted', label='estimated')
plt.xlabel('Date')
plt.xticks(rotation = 30)
plt.plot(better_df_predict['Date'], better_df_predict[sub_flow], linestyle = 'dotted', label='measured')
plt.ylabel('Estimated flow (cms)')
plt.legend()
plt.title('Better')
plt.savefig(save_path + 'Est. vs measured flow - ' + sub + ' better.png')
plt.show()

plt.boxplot(better_df_predict['error (%)'])
plt.ylabel('Error in est. flow (%)')
plt.title('Better')
plt.savefig(save_path + 'Est. error - ' + sub + ' better.png')
plt.show()

print('Better variables: ', better_var)
print('Better r2: ', better_r2)
print('Better accuracy: ', better_accuracy, '%')
print('Better avg error: ', better_avg_error, '%')
print('Better median error: ', better_med_error, '%')

print('BEST')

plt.plot(best_df_predict['Date'], best_df_predict['est. flow'], linestyle = 'dotted', label='estimated')
plt.xlabel('Date')
plt.xticks(rotation = 30)
plt.plot(best_df_predict['Date'], best_df_predict[sub_flow], linestyle = 'dotted', label='measured')
plt.ylabel('Est. flow (cms)')
plt.legend()
plt.title('Best')
plt.savefig(save_path + 'Est. vs measured flow - ' + sub + ' best.png')
plt.show()

plt.boxplot(best_df_predict['error (%)'])
plt.ylabel('Error in est. flow (%)')
plt.title('Best')
plt.savefig(save_path + 'Est. error - ' + sub + ' best.png')
plt.show()

print('Best variables: ', best_var)
print('Best r2: ', best_r2)
print('Best accuracy: ', best_accuracy, '%')
print('Best avg error: ', best_avg_error, '%')
print('Best median error: ', best_med_error, '%')

print('THE END')    

# export good, better, and best dataframes to Excel
writer = pd.ExcelWriter(save_path + 'Estimated daily flow 2018 - sub' + sub + ' good.xlsx', engine='xlsxwriter')
good_df_predict.to_excel(writer, index=False)
worksheet = writer.sheets['Sheet1']
worksheet.write('S2', 'Combo no.')
worksheet.write('T2', combo_no)
worksheet.write('S3', 'R2')
worksheet.write('T3', good_r2)
worksheet.write('S4', 'Accuracy, %')
worksheet.write('T4', good_accuracy)
worksheet.write('S5', 'Avg error, %')
worksheet.write('T5', str(good_avg_error))
worksheet.write('S6', 'Median error, %')
worksheet.write('T6', good_med_error)
for column in good_df_predict:
    column_length = max(good_df_predict[column].astype(str).map(len).max(), len(column))
    col_idx = good_df_predict.columns.get_loc(column)
    worksheet.set_column(col_idx, col_idx, column_length)
worksheet.set_column('A:A', 20)
worksheet.set_column('S:S', 15)
worksheet.insert_image('S7', save_path + 'Est. vs measured flow - ' + sub + ' good.png')
worksheet.insert_image('S27', save_path + 'Est. error - ' + sub + ' good.png')
writer.save()

writer = pd.ExcelWriter(save_path + 'Estimated daily flow 2018 - sub' + sub + ' better.xlsx', engine='xlsxwriter')
better_df_predict.to_excel(writer, index=False)
worksheet = writer.sheets['Sheet1']
worksheet.write('S2', 'Combo no.')
worksheet.write('T2', combo_no)
worksheet.write('S3', 'R2')
worksheet.write('T3', better_r2)
worksheet.write('S4', 'Accuracy, %')
worksheet.write('T4', better_accuracy)
worksheet.write('S5', 'Avg error, %')
worksheet.write('T5', str(better_avg_error))
worksheet.write('S6', 'Median error, %')
worksheet.write('T6', better_med_error)
for column in better_df_predict:
    column_length = max(better_df_predict[column].astype(str).map(len).max(), len(column))
    col_idx = better_df_predict.columns.get_loc(column)
    worksheet.set_column(col_idx, col_idx, column_length)
worksheet.set_column('A:A', 20)
worksheet.set_column('S:S', 15)
worksheet.insert_image('S7', save_path + 'Est. vs measured flow - ' + sub + ' better.png')
worksheet.insert_image('S27', save_path + 'Est. error - ' + sub + ' better.png')
writer.save()

writer = pd.ExcelWriter(save_path + 'Estimated daily flow 2018 - sub' + sub + ' best.xlsx', engine='xlsxwriter')
best_df_predict.to_excel(writer, index=False)
worksheet = writer.sheets['Sheet1']
worksheet.write('S2', 'Combo no.')
worksheet.write('T2', combo_no)
worksheet.write('S3', 'R2')
worksheet.write('T3', best_r2)
worksheet.write('S4', 'Accuracy, %')
worksheet.write('T4', best_accuracy)
worksheet.write('S5', 'Avg error, %')
worksheet.write('T5', str(best_avg_error))
worksheet.write('S6', 'Median error, %')
worksheet.write('T6', best_med_error)
for column in best_df_predict:
    column_length = max(best_df_predict[column].astype(str).map(len).max(), len(column))
    col_idx = best_df_predict.columns.get_loc(column)
    worksheet.set_column(col_idx, col_idx, column_length)
worksheet.set_column('A:A', 20)
worksheet.set_column('S:S', 15)
worksheet.insert_image('S7', save_path + 'Est. vs measured flow - ' + sub + ' best.png')
worksheet.insert_image('S27', save_path + 'Est. error - ' + sub + ' best.png')
writer.save()

In [ ]:
print(best_var)

### Below is a more detailed approach by testing all possible unique combinations of variables, but requires way longer processing time

In [ ]:
from itertools import combinations

# how many variables to include in each combination set? 
min_combo_no = 1
max_combo_no = len(all_variables)
total_combo = 0

for combo_no in range (min_combo_no, max_combo_no):
    combo = [",".join(map(str, comb)) for comb in combinations(all_variables, combo_no)]
    
    # probably need another loop function to go through each combination set in the list
    # insert machine learning and other data processing codes here
    
    # print the number of combination sets in each list
    #print(len(combo))
    total_combo = total_combo + len(combo)

print('total possible unique combinations: ', total_combo)

### Using random grid model to improve base model

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# the values below are somewhat selected arbitarily to cover a wide range of parameters

# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
#max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)

# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
#bootstrap = [True, False]

# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}

#pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
#rf = ske.RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = 1)
# Fit the random search model
#rf_random.fit(X_train, Y_train)

In [ ]:
#Y_pred = rf_random.predict(X_test)

#print('training period explained variance:', explained_variance_score(Y_test, Y_pred))
#print('training period mean abs error:',mean_absolute_error(Y_test, Y_pred))
#print('training period mean squared error:',mean_squared_error(Y_test, Y_pred))
#print('training period r2:',r2_score(Y_test, Y_pred))

In [ ]:
#def evaluate(model, X_train, Y_train):
#    predictions = model.predict(X_train)
#    errors = abs(predictions - Y_train)
#    mape = 100 * np.mean(errors / Y_train)
#    accuracy = 100 - mape
#    print('Model Performance')
#    print('Average Error: {:0.4f} cms.'.format(np.mean(errors)))
#    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
#    return accuracy

#class color:
#    BLUE = '\033[94m'
#    BOLD = '\033[1m'
#    END = '\033[0m'

#print(color.BOLD + color.BLUE + 'For Base Model' + color.END)
#base_accuracy = evaluate(rf_base, X_train, Y_train)

#print(color.BOLD + color.BLUE + 'For Random Model' + color.END)
#best_random = rf_random.best_estimator_
#random_accuracy = evaluate(best_random, X_train, Y_train)

#print(color.BOLD + color.BLUE + 'Base vs random model comparison' + color.END)
#print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

#pprint(rf_random.best_params_)

In [ ]:
#from sklearn.model_selection import GridSearchCV

# if the random model performed better than the base model, then use the best random grid to create range of each hyperparameter

# Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
#max_features = ['sqrt']

# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)

# Minimum number of samples required to split a node
#min_samples_split = [2, 3, 4]

# Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 3]

# Method of selecting samples for training each tree
#bootstrap = [False]

# Create the better grid
#better_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}

# We used randamized search to identify the better grid (i.e., narrow down the range for each hyperparameter)
# Now we use the better grid to instantiate the grid search model
#grid_search = GridSearchCV(estimator = rf, param_grid = better_grid, 
#                          cv = 3, n_jobs = 1, verbose = 2)


In [ ]:
# Fit the grid search to the data
#grid_search.fit(X_train, Y_train)
#grid_search.best_params_

#best_grid = grid_search.best_estimator_
#grid_accuracy = evaluate(best_grid, X_train, Y_train)

#print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

In [ ]:
#Y_pred = rf_random.predict(X_test)

#print('training period explained variance:', explained_variance_score(Y_test, Y_pred))
#print('training period mean abs error:',mean_absolute_error(Y_test, Y_pred))
#print('training period mean squared error:',mean_squared_error(Y_test, Y_pred))
#print('training period r2:',r2_score(Y_test, Y_pred))

## Predict 2018 data, then compare against actual sample avg flow data

In [ ]:
#df_test = df_weather[df_weather["Year"].isin([2018])]
#df_test = df_test.reset_index(drop=True)
#df_test = pd.merge(df_weather, df_testraw, left_on='Date', right_on='Sample date', how='inner')

# convert Sample date from timestamp to numeric because sklearn cannot process timestamp format
#df_test['Date'] = pd.to_numeric(pd.to_datetime(df_test['Date']))
#df_test = df_test[variables]

# use base or random depending on model performance
#Y_pred = rf_base.predict(df_test)
#Y_pred = rf_random.predict(df_test)

#df_predict = pd.DataFrame(Y_pred, columns=['flow_pred'])
#df_predict = pd.merge(df_test, df_predict, left_index=True, right_index=True)

# this is flow-weighted sample avg flow
#df_actual = pd.read_excel('Test data 2018.xlsx', sheet_name = 'By subwatershed')
#df_actual = df_actual[(df_actual['Site'] == 'Sub8') & (df_actual['Sample type'] == 'Base')]
#df_actual = df_actual[['Sample date','Flow (cms)']]
#df_actual = df_actual.dropna()

#df_predict = pd.merge(df_actual, df_predict['flow_pred'], left_index=True, right_index=True)
#df_predict['error (%)'] = round(((df_predict['Flow (cms)']-df_predict['flow_pred'])/df_predict['Flow (cms)']*100), 0)

#df_predict['Sample date'] = pd.to_datetime(df_predict['Sample date'])

#print('prediction period explained variance:',explained_variance_score(df_predict['Flow (cms)'], df_predict['flow_pred']))
#print('prediction period mean abs error:',mean_absolute_error(df_predict['Flow (cms)'], df_predict['flow_pred']))
#print('prediction period mean squared error:',mean_squared_error(df_predict['Flow (cms)'], df_predict['flow_pred']))
#print('prediction period r2:',r2_score(df_predict['Flow (cms)'], df_predict['flow_pred']))

#plt.scatter(df_predict['Flow (cms)'], df_predict['flow_pred'])
#plt.xlabel('Actual flow (cms)')
#plt.ylabel('Predicted flow (cms)')

In [ ]:
#writer = pd.ExcelWriter('predict flow 2018.xlsx')
# write dataframe to excel
#df_predict.to_excel(writer)
# save the excel
#writer.save()